In [ ]:
from __future__ import print_function

import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
import pandas as pd
import tensorflow as tf
from tensorflow.compat.v1.keras.models import save_model

import keras
import h5py
from keras import initializers
from keras.models import Sequential
from keras.models import Model
# from keras.models import load_model
from tensorflow.keras.models import load_model
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.layers import Input, InputLayer
from keras.layers import Embedding, Activation, Dropout, Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
from keras.layers.merge import Concatenate
# from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import multi_gpu_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, binarize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from collections import defaultdict

import re
import os
from os import listdir
# !pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import itertools
import operator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as mtick
!pip install wget
import wget
!pip install Path

In [18]:
AWS_Path = 'https://kstonedev.s3-us-west-2.amazonaws.com/W266/USPTO-2M/'
all_files = [
    '2006_USPTO.json',
    '2007_USPTO.json',
    '2008_USPTO.json',
    '2009_USPTO.json',
    '2010_USPTO.json',
    '2011_USPTO.json',
    '2012_USPTO.json',
    '2013_USPTO.json',
    '2014_USPTO.json',
    '2015_USPTO.json'
]

for file in all_files:
    wget.download(AWS_Path + file, out='.')
    print(file)

2006_USPTO.json
2007_USPTO.json
2008_USPTO.json
2009_USPTO.json
2010_USPTO.json
2011_USPTO.json
2012_USPTO.json
2013_USPTO.json
2014_USPTO.json
2015_USPTO.json


In [43]:
num_files_to_read = 2 # set to "all" or a number such as 2
if num_files_to_read == "all":
  print('Ingesting the following files:\n', sorted(all_files))
  patents = pd.concat(pd.read_json(f) for f in sorted(all_files))
else:
  print('Ingesting the following files:\n', sorted(all_files)[-num_files_to_read:])
  patents = pd.concat(pd.read_json(f) for f in sorted(all_files)[-num_files_to_read:])

Ingesting the following files:
 ['2014_USPTO.json', '2015_USPTO.json']


In [44]:
X = list(patents["Abstract"])
abstract_words = [x for sent in X for x in sent.split(' ')]
unique_words = len(set(abstract_words))
CRC_labels = patents.Subclass_labels
# binarize labels into 1-hot encodings
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(CRC_labels)
num_CRC_labels = len(y[0])

print("Total number of CRC labels", num_CRC_labels)

# save CRC label list to a file for inference
pd.DataFrame(mlb.classes_).to_csv('crc_labels.csv')

Total number of CRC labels 624


In [45]:
# Keep full patent table intact for analysis at end
P_train3, P_test3, y_train3, y_test3 = train_test_split(patents, y, test_size=0.20, random_state=42)
X_train3 = list(P_train3['Abstract'])
X_test3 = list(P_test3['Abstract'])

In [46]:
experiment_number = 3
if experiment_number == 1:
  (P_train, P_test, X_train, X_test, y_train, y_test) = (P_train1, P_test1, X_train1, X_test1, y_train1, y_test1)
elif experiment_number == 2:
  (P_train, P_test, X_train, X_test, y_train, y_test) = (P_train2, P_test2, X_train2, X_test2, y_train2, y_test2)
elif experiment_number == 3:
  (P_train, P_test, X_train, X_test, y_train, y_test) = (P_train3, P_test3, X_train3, X_test3, y_train3, y_test3)
elif experiment_number == 4:
  (P_train, P_test, X_train, X_test, y_train, y_test) = (P_train4, P_test4, X_train4, X_test4, y_train4, y_test4)
else:
  print("Unknown experiment number")

In [47]:
# Convert to word embeddings

# https://keras.io/preprocessing/text/
tokenizer = Tokenizer(num_words=unique_words)
tokenizer.fit_on_texts(X_train)

# Top 10 words
top10words = sorted(tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True)[:10]
# print("Top 10 words:\n", top10words)

# Convert text to sequence of numbers, each number representing a word
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [48]:
# Use GloVe word embeddings to convert text inputs to their numeric counterparts

embeddings_dictionary = dict()

glove_file = open('glove.6B.100d.txt', encoding='utf8')

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [49]:
# metric f1 definition
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
def weighted_bce(y_true, y_pred):
    # weights become 2 if y_true is 1, and 1 if y_true is 0
    weights = (y_true * 2.) + (1. - y_true)
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    return weighted_bce

In [50]:
filters = 512
hidden_dims = 512
print('Build model...')
input = keras.Input(shape=(maxlen,), name='input_embeddings')
embedding = keras.layers.Embedding(vocab_size, 100,
                    input_length=maxlen, weights=[embedding_matrix],
                                             trainable=False)(input)
conv0 = keras.layers.Conv1D(filters, 2, activation='relu')(embedding)
maxpool0 = keras.layers.GlobalMaxPooling1D()(conv0)
conv1 = keras.layers.Conv1D(filters, 3, activation='relu')(embedding)
maxpool1 = keras.layers.GlobalMaxPooling1D()(conv1)
conv2 = keras.layers.Conv1D(filters, 4, activation='relu')(embedding)
maxpool2 = keras.layers.GlobalMaxPooling1D()(conv2)
conv3 = keras.layers.Conv1D(filters, 5, activation='relu')(embedding)
maxpool3 = keras.layers.GlobalMaxPooling1D()(conv3)
concat1 = keras.layers.concatenate([maxpool0, maxpool1, maxpool2, maxpool3], axis=1)
dropout1 = keras.layers.Dropout(rate=0.2)(concat1)
dense = keras.layers.Dense(hidden_dims, activation='relu', name='dense')(dropout1)
batchnorm = keras.layers.BatchNormalization()(dense)
dense = keras.layers.Dropout(rate=0.5)(batchnorm)
pred = keras.layers.Dense(num_CRC_labels, activation='sigmoid', name='crc')(dense)
model = keras.models.Model(inputs=input, outputs=pred)

Build model...


InternalError: Dst tensor is not initialized.
	 [[{{node _arg_embedding_3/Placeholder_0_0}}]]

In [ ]:
model.summary()
model.compile(loss=weighted_bce,
              optimizer='adam',
              metrics=[f1,
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Precision(name='precision_1', top_k=1),
                      tf.keras.metrics.Recall(name='recall'),
                      tf.keras.metrics.Recall(name='recall_5', top_k=5)])


In [ ]:
history3 = model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, y_test))

In [51]:
keras.__version__

'2.2.4'

In [53]:
tf.__version__

'1.13.1'

In [54]:
!python --version

Python 3.5.2
